In [3]:
# Import necessary libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from Network import Network
from Tools import generate_mnist, createPath, initDataframe, updateDataframe
from simulated_sampler import SimulatedAnnealingSampler

# Set up the arguments for the training
class Args:
    dataset = 'mnist'
    simulated = 1  # Use simulated annealing
    mode = 'ising'
    layersList = [784, 120, 40]  # Input, Hidden, Output layers
    n_iter_free = 10
    n_iter_nudge = 10
    frac_anneal_nudge = 0.25
    N_data = 1000  # Number of training data points
    N_data_test = 100  # Number of testing data points
    beta = 5
    batch_size = 1
    lrW0 = 0.01  # Learning rate for input-hidden weights
    lrW1 = 0.01  # Learning rate for hidden-output weights
    lrB0 = 0.001  # Learning rate for hidden biases
    lrB1 = 0.001  # Learning rate for output biases
    epochs = 50
    load_model = 0
    gain_weight0 = 0.5
    gain_weight1 = 0.25
    bias_lim = 4.0
    chain_strength = 1.0
    auto_scale = 0

args = Args()

# Load MNIST data
train_loader, test_loader, _ = generate_mnist(args)

# Initialize the network
net = Network(args)

# Initialize the sampler
simu_sampler = SimulatedAnnealingSampler()

# Set up the base path and dataframe for saving results
BASE_PATH = createPath(args)
dataframe = initDataframe(BASE_PATH)

# Training and testing loops
for epoch in tqdm(range(args.epochs)):
    # Train the network for one epoch
    exact_loss, exact_pred, qpu_loss, qpu_pred = train(net, args, train_loader, simu_sampler, None, None)
    
    # Test the network after each epoch
    exact_loss_test, exact_pred_test, qpu_loss_test, qpu_pred_test = test(net, args, test_loader, simu_sampler, None, None)
    
    # Update the results dataframe
    dataframe = updateDataframe(
        BASE_PATH, dataframe,
        exact_pred / len(train_loader.dataset) * 100,
        exact_pred_test / len(test_loader.dataset) * 100,
        qpu_pred / len(train_loader.dataset) * 100,
        qpu_pred_test / len(test_loader.dataset) * 100,
        exact_loss, exact_loss_test, qpu_loss, qpu_loss_test
    )

    # Save the model after each epoch
    torch.save(net.state_dict(), f"{BASE_PATH}/model_epoch_{epoch}.pth")

# Plotting results
from plotFunction import plot_mean_error, plot_mean_loss

plot_mean_error([dataframe['QPU_Train_Error']], [dataframe['QPU_Test_Error']], [], [])
plot_mean_loss([dataframe['QPU_Train_Loss']], [dataframe['QPU_Test_Loss']])

plt.show()


AttributeError: 'Args' object has no attribute 'data_augmentation'

In [7]:
import sys
print(sys.executable)


c:\Users\Valley\AppData\Local\Programs\Python\Python310\python.exe
